# Advent of Code 2022

## Day 11: Monkey in the Middle

Solution code by [leechristie](https://github.com/leechristie) for Advent of Code 2022.

Today's part 2 required a bit of maths rather than just blind coding! I think that's the first time in this year's AoC that needed what I'd describe as mathematical thinking. But not too difficult, so I still managed to solve it myself.

Firstly, in part 1, I build a `Monkey` class and a generator which loops over the file yielding `Monkey` objects to load the data. I used this as mutable state to track the state of the monkeys. The list of items is a `deque` although I'm sure a `list` would have worked fine. There's a lot of printing code in my logic behind this `verbos=` flag that gets passed to methods fo I could compare against the sample output in the question.

I added a progress bar with `tqdm` to see how long part 2 was going to take without modification to the code other than removing the divide-by-3 and setting the rounds to 10,000. It would take too long. There item worry numbers are too big to compute even with Python's unbounded ints.

I modified the method which applies the operator to divide by the product of all monkey divisors. They are all primes. I assume this is the lowest base I can use inless there's some other trick I'm missing? This works first time!

Very fun.

### Update!

I've refactored my solution a bit

Now, instead of patching in the `% N` thing, I pass in a `Callable` called `worry_reduction` which gets applied as `lambda item: item // 3` in part 1 and `lambda item: item % N` in part 2. That way the code can be the same for both parts except for that and the change to the number of round!

I also removed the `verbose=` stuff so there's no longer the extra printing that's not really needed, allowing me to make other small clean ups.

### Imports

In [ ]:
from typing import Iterator, Union, Callable
from collections import deque

### Data File Processing

In [ ]:
def read_data_blocks(filename: str) -> Iterator[list[str]]:

    buffer = []

    with open(filename) as file:

        for line in file:

            line = line.strip('\n')
            if line:
                buffer.append(line)
            else:
                yield buffer
                buffer = []

    # in case the final block is not terminated with \n
    if buffer:
        yield buffer

In [ ]:
def strip_line(line: str, start: str, end: str='') -> str:
    assert line.startswith(start), f'line "{line}" does not start with "{start}"'
    assert line.endswith(end), f'line "{line}" does not end with "{end}"'
    if end:
        return line[len(start):-len(end)]
    return line[len(start):]

### Monkey Object

In [ ]:
class Monkey:

    __slots__ = ['number', 'items', 'operator', 'operand', 'divisor', 'if_true', 'if_false', 'inspections']

    def __init__(self,
                 number: int,
                 items: list[int],
                 operator: str,
                 operand: Union[str, int],
                 divisor: int,
                 if_true: int,
                 if_false: int):
        self.number = number
        self.items = deque(items)
        self.operator = operator
        self.operand = operand
        self.divisor = divisor
        self.if_true = if_true
        self.if_false = if_false
        self.inspections = 0

    def apply_operator(self, old: int) -> int:
        self.inspections += 1
        if self.operator == '*':
            return old * (old if self.operand == 'old' else self.operand)
        return old + (old if self.operand == 'old' else self.operand)

In [ ]:
def read_monkeys(filename: str) -> list[Monkey]:

    rv = []

    for block in read_data_blocks(filename):

        assert len(block) == 6
        monkey_number, starting, op, test, if_true, if_false = block

        monkey_number = int(strip_line(monkey_number, 'Monkey ', ':'))
        starting = strip_line(starting, '  Starting items: ')
        starting = [int(e) for e in starting.split(', ')]
        op = strip_line(op, '  Operation: new = old ')
        operator, operand = op.split(' ')
        try:
            operand = int(operand)
        except ValueError:
            pass
        test = int(strip_line(test, '  Test: divisible by '))
        if_true = int(strip_line(if_true, '    If true: throw to monkey '))
        if_false = int(strip_line(if_false, '    If false: throw to monkey '))

        assert monkey_number == len(rv)
        rv.append(Monkey(monkey_number, starting, operator, operand, test, if_true, if_false))

    return rv

### Processing Rounds of Monkey Business

In [ ]:
def monkey_round(monkeys: list[Monkey],
                 worry_reduction: Callable[[int], int]) -> None:

    for monkey in monkeys:

        while monkey.items:

            item = monkey.items.popleft()
            item = monkey.apply_operator(item)

            item = worry_reduction(item)

            if item % monkey.divisor == 0:
                monkeys[monkey.if_true].items.append(item)
            else:
                monkeys[monkey.if_false].items.append(item)

In [ ]:
def do_all_rounds(monkeys: list[Monkey],
                  num_rounds: int,
                  worry_reduction: Callable[[int], int]) -> int:

    for rounds_complete in range(num_rounds):
        monkey_round(monkeys, worry_reduction=worry_reduction)

    inspections = []
    for monkey in monkeys:
        inspections.append(monkey.inspections)
    inspections.sort(reverse=True)
    monkey_business = inspections[0] * inspections[1]

    return monkey_business

### Part 1

In [ ]:
INPUT_FILE = 'data/input11.txt'

In [ ]:
def main():

    monkeys = read_monkeys(INPUT_FILE)
    num_rounds = 20
    worry_reduction = lambda item: item // 3

    monkey_business = do_all_rounds(monkeys, num_rounds, worry_reduction)
    print(f'The level of monkey business after {num_rounds} rounds is {monkey_business}')

In [ ]:
if __name__ == '__main__':
    main()

### Part 2

In [ ]:
def product_of_divisors(monkeys: list[Monkey]) -> int:
    rv = 1
    for m in monkeys:
        rv *= m.divisor
    return rv

In [ ]:
def main():

    monkeys = read_monkeys(INPUT_FILE)
    num_rounds = 10000
    base = product_of_divisors(monkeys)
    worry_reduction = lambda item: item % base

    monkey_business = do_all_rounds(monkeys, num_rounds, worry_reduction)
    print(f'The level of monkey business after {num_rounds} rounds is {monkey_business}')

In [ ]:
if __name__ == '__main__':
    main()